In [1]:
import great_expectations as gx
import pandas as pd

# Initialize an in-memory DataContext
context = gx.get_context()

# Create a sample DataFrame
data = {'value': [1, 2, None, 4, 5]}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)

# Define the batch request
batch_request = {
    "datasource_name": "my_pandas_datasource",
    "data_connector_name": "default_runtime_data_connector_name",
    "data_asset_name": "data.csv",
    "runtime_parameters": {"path": "data.csv"},
    "batch_identifiers": {"default_identifier_name": "default_identifier"},
}

# Define the suite name
suite_name = "expect_no_missing_values"

# Create or load the expectation suite manually if it does not exist
try:
    context.get_expectation_suite(suite_name)
except gx.exceptions.exceptions.DataContextError:
    # If not found, create a new expectation suite
    context.add_expectation_suite(expectation_suite_name=suite_name)

# Create a Validator for the batch
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=suite_name
)

# Add an expectation to check that the 'value' column has no missing values
validator.expect_column_to_not_be_null("value")

# Save the expectation suite (in-memory)
context.save_expectation_suite(validator.expectation_suite)

# Run a checkpoint to validate the data
checkpoint = gx.checkpoint.Checkpoint(
    name="missing_value_check",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": suite_name
        }
    ]
)

# Execute the checkpoint
result = checkpoint.run()

# Print the validation results
import pprint
pprint.pprint(result.to_json_dict())


AttributeError: 'EphemeralDataContext' object has no attribute 'get_expectation_suite'